In [2]:
import torch
from torch import nn
import torch.nn.functional as F

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
with open('../Data/shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [6]:
type(text)

str

In [8]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [9]:
len(text)

5445609

In [14]:
text[300:350]

'l,\n  Making a famine where abundance lies,\n  Thy s'

In [15]:
all_characters = set(text)

In [16]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [17]:
len(all_characters)

84

In [19]:
for pair in enumerate(all_characters):
    print(pair)

(0, ')')
(1, '8')
(2, 'J')
(3, 'V')
(4, '}')
(5, 'X')
(6, 't')
(7, '4')
(8, 'h')
(9, 'x')
(10, 'b')
(11, 'I')
(12, 'Y')
(13, '9')
(14, '`')
(15, 'e')
(16, '-')
(17, 'E')
(18, '[')
(19, 'c')
(20, ']')
(21, 'a')
(22, '!')
(23, 'S')
(24, 'D')
(25, '\n')
(26, 'B')
(27, 'H')
(28, '"')
(29, '|')
(30, ';')
(31, '5')
(32, 'g')
(33, 'm')
(34, 'U')
(35, 'W')
(36, '2')
(37, '3')
(38, 'j')
(39, '_')
(40, '6')
(41, 'n')
(42, 's')
(43, '0')
(44, 'p')
(45, 'q')
(46, 'f')
(47, 'C')
(48, 'i')
(49, ',')
(50, ':')
(51, '.')
(52, 'z')
(53, '<')
(54, 'A')
(55, 'u')
(56, '1')
(57, 'L')
(58, 'k')
(59, 'K')
(60, 'P')
(61, '>')
(62, '&')
(63, 'd')
(64, 'O')
(65, 'T')
(66, 'o')
(67, 'N')
(68, 'F')
(69, 'r')
(70, "'")
(71, 'M')
(72, 'G')
(73, 'R')
(74, '?')
(75, 'w')
(76, 'Q')
(77, '7')
(78, '(')
(79, 'Z')
(80, 'v')
(81, ' ')
(82, 'l')
(83, 'y')


In [20]:
# num --> letter
decoder = dict(enumerate(all_characters))

In [21]:
# letter --> num
encoder = {char: ind for ind, char in decoder.items()}

In [22]:
encoder

{')': 0,
 '8': 1,
 'J': 2,
 'V': 3,
 '}': 4,
 'X': 5,
 't': 6,
 '4': 7,
 'h': 8,
 'x': 9,
 'b': 10,
 'I': 11,
 'Y': 12,
 '9': 13,
 '`': 14,
 'e': 15,
 '-': 16,
 'E': 17,
 '[': 18,
 'c': 19,
 ']': 20,
 'a': 21,
 '!': 22,
 'S': 23,
 'D': 24,
 '\n': 25,
 'B': 26,
 'H': 27,
 '"': 28,
 '|': 29,
 ';': 30,
 '5': 31,
 'g': 32,
 'm': 33,
 'U': 34,
 'W': 35,
 '2': 36,
 '3': 37,
 'j': 38,
 '_': 39,
 '6': 40,
 'n': 41,
 's': 42,
 '0': 43,
 'p': 44,
 'q': 45,
 'f': 46,
 'C': 47,
 'i': 48,
 ',': 49,
 ':': 50,
 '.': 51,
 'z': 52,
 '<': 53,
 'A': 54,
 'u': 55,
 '1': 56,
 'L': 57,
 'k': 58,
 'K': 59,
 'P': 60,
 '>': 61,
 '&': 62,
 'd': 63,
 'O': 64,
 'T': 65,
 'o': 66,
 'N': 67,
 'F': 68,
 'r': 69,
 "'": 70,
 'M': 71,
 'G': 72,
 'R': 73,
 '?': 74,
 'w': 75,
 'Q': 76,
 '7': 77,
 '(': 78,
 'Z': 79,
 'v': 80,
 ' ': 81,
 'l': 82,
 'y': 83}

In [23]:
encoded_text = np.array([encoder[char] for char in text])

In [25]:
encoded_text[:1000]

array([25, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81,
       81, 81, 81, 81, 81, 56, 25, 81, 81, 68, 69, 66, 33, 81, 46, 21, 48,
       69, 15, 42,  6, 81, 19, 69, 15, 21,  6, 55, 69, 15, 42, 81, 75, 15,
       81, 63, 15, 42, 48, 69, 15, 81, 48, 41, 19, 69, 15, 21, 42, 15, 49,
       25, 81, 81, 65,  8, 21,  6, 81,  6,  8, 15, 69, 15, 10, 83, 81, 10,
       15, 21, 55,  6, 83, 70, 42, 81, 69, 66, 42, 15, 81, 33, 48, 32,  8,
        6, 81, 41, 15, 80, 15, 69, 81, 63, 48, 15, 49, 25, 81, 81, 26, 55,
        6, 81, 21, 42, 81,  6,  8, 15, 81, 69, 48, 44, 15, 69, 81, 42,  8,
       66, 55, 82, 63, 81, 10, 83, 81,  6, 48, 33, 15, 81, 63, 15, 19, 15,
       21, 42, 15, 49, 25, 81, 81, 27, 48, 42, 81,  6, 15, 41, 63, 15, 69,
       81,  8, 15, 48, 69, 81, 33, 48, 32,  8,  6, 81, 10, 15, 21, 69, 81,
        8, 48, 42, 81, 33, 15, 33, 66, 69, 83, 50, 25, 81, 81, 26, 55,  6,
       81,  6,  8, 66, 55, 81, 19, 66, 41,  6, 69, 21, 19,  6, 15, 63, 81,
        6, 66, 81,  6,  8

In [26]:
decoder[27]

'H'

In [27]:
decoder[81]

' '

In [34]:
def one_hot_encoder(encoded_text, num_uni_chars):

    # encoded_text --> catch of encoder text
    # num_uni_chars --> len(set(text))

    # create place holder
    one_hot = np.zeros((encoded_text.size, num_uni_chars))

    # need to have float32 to make sure no errors using pytorch later
    one_hot = one_hot.astype(np.float32)

    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0

    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))

    return one_hot

In [35]:
arr = np.array([1, 2, 0])

In [36]:
arr

array([1, 2, 0])

In [37]:
one_hot_encoder(arr, 3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [38]:
example_text = np.arange(10)

In [39]:
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [40]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [60]:
# Here create a generator, which use yield keyword, not save anything in memory, but generate batches requested

def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):

    # X: encoded text of length seq_len
    # Y: encoded text shifted by one

    # the total number of chars used per batch:
    char_per_batch = samp_per_batch * seq_len

    # calculate the number of batches available to make, given the length of encoded text
    num_batches_avail = int(len(encoded_text)/char_per_batch)

    # Cut off the end of the encoded text that won't fit evenly into a batch
    # this may cause loss of the some information at the end of text, but as long as your text is long enough, it will not affect your results.
    encoded_text = encoded_text[:num_batches_avail*char_per_batch]

    encoded_text = encoded_text.reshape((samp_per_batch, -1))

    # for each row in the array, graph the feature chars and save y as the chars shift by one
    for n in range(0, encoded_text.shape[1], seq_len):

        x = encoded_text[:, n:n+seq_len]

        # create zeros array to the same shape as x
        y = np.zeros_like(x)

        try:

            y[:,:-1] = x[:,1:]
            y[:,-1] = encoded_text[:, n+seq_len]

        # at the very end, out of range index error
        except:
            y[:,:-1] = x[:,1:]
            y[:,-1] = encoded_text[:,0]

        # yield x,y in a tuple
        yield x, y

In [61]:
sample_text = encoded_text[:20]

In [62]:
sample_text

array([25, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81,
       81, 81, 81])

In [63]:
len(sample_text)

20

In [64]:
batch_generator = generate_batches(sample_text, samp_per_batch=2, seq_len=5)

In [65]:
x, y =next(batch_generator)

In [66]:
x

array([[25, 81, 81, 81, 81],
       [81, 81, 81, 81, 81]])

In [67]:
y

array([[81, 81, 81, 81, 81],
       [81, 81, 81, 81, 81]])

In [68]:
sample_text = np.arange(20)

In [69]:
sample_text

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [70]:
batch_generator = generate_batches(sample_text, samp_per_batch=2, seq_len=5)

In [71]:
x, y = next(batch_generator)

In [72]:
x

array([[ 0,  1,  2,  3,  4],
       [10, 11, 12, 13, 14]])

In [73]:
y

array([[ 1,  2,  3,  4,  5],
       [11, 12, 13, 14, 15]])

## Create LSTM model

In [96]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden=256, num_layers=4, drop_prob=0.5, device='cpu'):

        super().__init__()

        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.device = device  # set device (either 'cuda' or 'cpu')

        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char:ind for ind, char in decoder.items()}

        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))

    def forward(self, x, hidden):

        lstm_output, hidden = self.lstm(x, hidden)

        drop_output = self.dropout(lstm_output)

        drop_output = drop_output.contiguous().view(-1, self.num_hidden)

        final_out = self.fc_linear(drop_output)
        return final_out, hidden

    def hidden_state(self,batch_size):
        # Initialize the hidden state tensors on the correct device
        hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).to(self.device),
                  torch.zeros(self.num_layers, batch_size, self.num_hidden).to(self.device))

        return hidden

In [97]:
# Detect device (either 'cuda' if GPU is available or 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [99]:
# Instantiate the model and move it to the approciate device
model = CharModel(all_chars=all_characters,
                  num_hidden=512,
                  num_layers=3,
                  drop_prob=0.5,
                  device=device)

In [100]:
model = model.to(device)

In [76]:
total_param = []

for p in model.parameters():
    total_param.append(int(p.numel()))

In [77]:
sum(total_param)

5470292

In [78]:
# Loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [79]:
train_percent = 0.1

In [80]:
train_ind = int(len(encoded_text)*train_percent)

In [81]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

In [82]:
len(train_data)

544560

In [83]:
len(val_data)

4901049

In [85]:
train_percent = 0.9

In [86]:
train_ind = int(len(encoded_text)*train_percent)

In [87]:
train_data = encoded_text[:train_ind]

In [88]:
val_data = encoded_text[train_ind:]

## Variables

In [101]:
# Set hyperparameters
epochs = 60
batch_size = 100

seq_len = 100

tracker = 0
num_char = max(encoded_text)+1

In [102]:
model.train()

for i in range(epochs):

    hidden = model.hidden_state(batch_size)  # Initialize hidden state

    for x, y in generate_batches(train_data, batch_size, seq_len):

        tracker += 1

        # Convert input to one-hot and then to torch tensors
        x = one_hot_encoder(x, num_char)

        inputs = torch.from_numpy(x).float().to(device)  # move input to device
        targets = torch.from_numpy(y).long().to(device)  # move targets to device

        # Detach hidden state for the next batch to avoid backpropagating through the entire sequence
        hidden = tuple([state.data for state in hidden])

        # Zero the gradients
        model.zero_grad()

        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())

        # Backward pass and update parameters
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

        if tracker % 25 == 0:

            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()   # Set model to evaluation mode

            with torch.no_grad():  # Disable gradient calculation during validation
                for x, y in generate_batches(val_data, batch_size, seq_len):
    
                    x = one_hot_encoder(x, num_char)
    
                    inputs = torch.from_numpy(x).float().to(device)
                    targets = torch.from_numpy(y).long().to(device)
            
                    val_hidden = tuple([state.data for state in val_hidden])
    
                    lstm_output, val_hidden = model.forward(inputs, val_hidden)
                    val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
    
                    val_losses.append(val_loss.item())

            model.train()  # Switch back to training mode

            print(f'Epoch: {i} Step: {tracker} Val Loss: {val_loss.item()}')

Epoch: 0 Step: 25 Val Loss: 4.440383434295654
Epoch: 0 Step: 50 Val Loss: 4.440383434295654
Epoch: 0 Step: 75 Val Loss: 4.440383434295654
Epoch: 0 Step: 100 Val Loss: 4.440383434295654
Epoch: 0 Step: 125 Val Loss: 4.440383434295654


KeyboardInterrupt: 

In [103]:
# Save the model
model_name = 'hidden512_layers3_shakes.net'
torch.save(model.state_dict(), model_name)

## Load saved model for evaluation

In [147]:
model = CharModel(
    all_chars=all_characters, 
    num_hidden=512, 
    num_layers=4, 
    drop_prob=0.5, 
    device=device,
)

In [148]:
model_name = 'example.net'
model.load_state_dict(torch.load(model_name, map_location=torch.device('cpu')))
model.eval()

CharModel(
  (lstm): LSTM(84, 512, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

In [149]:
# Function to predict the next character
def predict_next_char(model, char, hidden=None, k=1):

    # Encode the input character
    encoded_text = model.encoder[char]

    encoded_text = np.array([[encoded_text]])

    # One-hot encode the input character
    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))

    # Convert to torch tensor and move ot the appropriate device
    inputs = torch.from_numpy(encoded_text).float().to(model.device)

    # detach hidden state to avoid backprop through entire history
    hidden = tuple([state.data for state in hidden])

    # Perform forward pass through the model
    lstm_out, hidden = model(inputs, hidden)

    # Get probabilities of the next character
    probs = F.softmax(lstm_out, dim=1).data

    # Move probabilities to cpu and process them as numpy arrays
    probs = probs.cpu()

    probs, index_positions = probs.topk(k)

    index_positions = index_positions.numpy().squeeze()

    probs = probs.numpy().flatten()

    # Normalize the probabilities
    probs = probs/probs.sum()

    # Randomly choose the next character based on the probabilities
    char = np.random.choice(index_positions, p=probs)

    # Return the decoded character and the new hidden state
    return model.decoder[char], hidden

In [150]:
# Function to generate next char
def generate_text(model, size, seed='The', k=1):

    model = model.to(model.device)  # Ensure the model is on the correct device

    # Set model to evaluation mode
    model.eval()

    # Initialize the output with the seed
    output_chars = [c for c in seed]

    # Initialize hidden state for batch size of 1
    hidden = model. hidden_state(1)

    # Prime the model with the seed next
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)

    output_chars.append(char)

    # Generate text of the specified size
    for i in range(size):

        char,hidden = predict_next_char(model, output_chars[-1], hidden, k=k)

        output_chars.append(char)

    return ''.join(output_chars)

In [142]:
# Example usage: generate 1000 characters of text
print(generate_text(model, 1000, seed='The ', k=3))

The VV>Vi>VVV>VV>Vi>>iVVVVV>VVViVV>>>>V>VV>>VVVVVVVV>VVVVi>>>VVV>>>ViVV>VVVVV>iiV>VV>ViVVVVV>V>V>>>>iiVViVViiVViV>VViV>VV>Vi>>V>V>iVV>VVVVViV>V>V>>V>>>VVVVVVVVVVVVVVVVV>VV>VVV>iViVVVVViV>ViV>ViVVVV>VVVVV>VVViVVViVVV>iVVVVVVVVVV>VVVVVV>iVV>VVVVVV>iiVV>VVVVVVV>VVV>V>Vi>>VVVVVV>Vi>VViV>VV>VVVVVVVViiiViVVVV>ViVVi>VVV>VVVV>>iVVViVV>iVV>VVVV>i>i>iVVV>VVVVViiVVi>>VVV>VV>V>VVVVVVVi>ViVVViVVVVVVVi>VV>VVViVViVV>VVViVVVVVVVViViV>>>V>VVV>VVVVVVVV>VVVVV>VV>V>>V>VVii>>>VV>>VVi>VViViV>VVViVViVViV>i>VVVVViiVViVVVVVi>VVi>VVi>VVViVVVVVViViVV>>VVVV>iVVi>VV>>>>>VVViiiV>V>VVVVVVVVVVVVVV>iiVVV>VVVVVV>V>V>VViVVV>iViiVVVV>V>>VVViVVVV>V>VVi>V>>VVVVVVV>VV>iVVVV>VVVVVVViV>VVVVii>VVVVVVVVV>VVViVVVVi>>VVV>VVViV>V>VVVi>>iVVVVVi>VVVVVVVViV>i>iiV>Vi>VVViVVVV>iVVViVVVVVViVVVVV>iiV>VV>>V>VVVVi>V>iVVVVVVV>VVVV>iVV>VV>VVVVViViViVV>>i>VVVVVVVVVV>iVV>VVVVVVV>VVVVVVVVi>i>VVViVVV>VViVVViVViVi>>VViVV>>iVVVi>Vi>iVViVVVV>>V>V>iVVV>>VV>Vii>ViVVV>V>iViViVViVVVVV>>>V>iVVV>ViVVVVVVVVVVVV>VV>>>VVVVVViVVV>VVVVVVViVV>iViVVVViVViViVVVV